<a href="https://colab.research.google.com/github/reic/colab_python/blob/main/%E9%8C%84%E9%9F%B3%E6%AA%94%E8%BD%89%E6%96%87%E5%AD%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 語音檔轉文字套件
1. 安裝 SpeechRecognition 套件(若已安裝可跳過)
2. 掛載 google drive
3. 設定環境變數

## 安裝語音轉文字套件

In [2]:
# 執行安裝套件
!pip3 install SpeechRecognition

     |████████████████████████████████| 32.8MB 132kB/s 


## 掛載 google 遠端硬碟

In [ ]:
# 掛載 Google drive
from google.colab import drive
drive.mount('/content/drive')

## 設定環境變數
 
1. 取得 mp3 檔路徑
2. 設定 wav 切割檔的輸出路徑
3. 設定 txt 的輸出路徑
4. 若 wav, txt 的目錄存在，是否移除重建。預設：是y

In [16]:
#@title 基礎環境設定
import os
import shutil
import speech_recognition as sr
import wave
import json
import numpy as np

#@markdown 取得 mp3 路徑
mp3Name= '/content/drive/MyDrive/210121_001.MP3' #@param {type:"string"}
#@markdown 將 mp3 轉換為 wav 檔，然後切割為 30 秒一個檔案，設定 wav 切割檔的存放目錄
wav_path='wav' #@param {type:"string"}
#@markdown 將30秒 wav 檔轉語音的文字檔存放目錄
txt_path='txt' #@param {type:"string"}

workpath=os.path.dirname(mp3Name)
mp3Name=os.path.basename(mp3Name)
FileName = mp3Name[:-4]+".wav"
os.chdir(workpath)
#@markdown 若 wav_path, txt_path 目錄存在是否移除重建
chk='y' #@param ["y","n"]

## 音頻轉換與切割

1. 將 mp3 轉成 wav 檔
2. 將音頻切割 30 秒一個檔案，並置於 wav_path 目錄下
3. 建立 txt_path ，做為語音判識的輸出檔




In [17]:
def reset_dir(path):
    try:
        os.mkdir(path)
    except Exception:
        shutil.rmtree(path)
        os.mkdir(path)
 
def CutFile(FileName, target_path):
 
    # print("CutFile File Name is ", FileName)
    f = wave.open(FileName, "rb")
    params = f.getparams()    
    nchannels, sampwidth, framerate, nframes = params[:4]
    CutFrameNum = framerate * CutTimeDef
    # 讀取格式資訊
    # 一次性返回所有的WAV檔案的格式資訊，它返回的是一個組元(tuple)：聲道數, 量化位數（byte    單位）, 採
    # 樣頻率, 取樣點數, 壓縮型別, 壓縮型別的描述。wave模組只支援非壓縮的資料，因此可以忽略最後兩個資訊

    # print("CutFrameNum=%d" % (CutFrameNum))
    # print("nchannels=%d" % (nchannels))
    # print("sampwidth=%d" % (sampwidth))
    # print("framerate=%d" % (framerate))
    # print("nframes=%d" % (nframes))

    str_data = f.readframes(nframes)
    f.close()  # 將波形資料轉換成陣列
    # Cutnum =nframes/framerate/CutTimeDef
    # 需要根據聲道數和量化單位，將讀取的二進位制資料轉換為一個可以計算的陣列
    wave_data = np.frombuffer(str_data, dtype=np.short)
    wave_data.shape = -1, 2
    wave_data = wave_data.T
    temp_data = wave_data.T
    # StepNum = int(nframes/200)
    StepNum = CutFrameNum
    StepTotalNum = 0
    haha = 0
    while StepTotalNum < nframes:
        # for j in range(int(Cutnum)):
        # print("Stemp=%d" % (haha))
        SaveFile = "%s-%03d.wav" % (FileName[:-4], (haha+1))
        # print(FileName)
        if haha % 3==0:
          print("*",end='')
        temp_dataTemp = temp_data[StepNum * (haha):StepNum * (haha + 1)]
        haha = haha + 1
        StepTotalNum = haha * StepNum
        temp_dataTemp.shape = 1, -1
        temp_dataTemp = temp_dataTemp.astype(np.short)  # 開啟WAV文件
        f = wave.open(target_path+"/" + SaveFile, "wb")
        # 配置聲道數、量化位數和取樣頻率
        f.setnchannels(nchannels)
        f.setsampwidth(sampwidth)
        f.setframerate(framerate)
        # 將wav_data轉換為二進位制資料寫入檔案
        f.writeframes(temp_dataTemp.tobytes())
        f.close()


print(" mp3 轉 wav 檔 ".center(100,'=')) 
os.system('{} -i {} {}'.format("ffmpeg",mp3Name, FileName))
print(" Wav 檔名為 {} ".format(FileName).center(96))
if chk == 'y':
    reset_dir(wav_path)
    reset_dir(txt_path)
# # Cut Wave Setting
CutTimeDef = 30  # 以1s截斷檔案
print(" 音頻以每{}秒分割 ".format(CutTimeDef).center(94,'='))
CutFile(FileName, wav_path)
print("")
print(" 完成分割 ".center(100,'-'))
os.remove(FileName)


=========================================== mp3 轉 wav 檔 ============================================
                                     Wav 檔名為 210121_001.wav                                     
========================================= 音頻以每30秒分割 ==========================================
*********************************************
----------------------------------------------- 完成分割 -----------------------------------------------


## 文字轉語音

In [21]:

def VoiceToText(path, files, target_path):
    for file in files:
        txt_file = "%s/%s.txt" % (target_path, file[:-4])
        if os.path.isfile(txt_file):
            continue
        with open("%s/%s.txt" % (target_path, file[:-4]), "w", encoding="utf-8") as f:
            f.write("%s:\n" % file)
            r = sr.Recognizer()  # 預設辨識英文
            with sr.WavFile(path+"/"+file) as source:  # 讀取wav檔
                audio = r.record(source)
            try:
                text = r.recognize_google(audio,language="zh-TW")
                print(file)
                if len(text) == 0:
                    print("===無資料==")
                    continue

                print(text)
                f.write("%s \n\n" % text)
                if file == files[-1]:
                    print("結束翻譯")
            except sr.RequestError as e:
                print("無法翻譯{0}".format(e))
                # 兩個 except 是當語音辨識不出來的時候 防呆用的
                # 使用Google的服務
            except LookupError:
                print("Could not understand audio")
            except sr.UnknownValueError:
                print("Error: 無法識別 Audio")


def texts_to_one(path, target_file):
    files = os.listdir(path)
    with open(target_file, "w", encoding="utf-8") as f:
        for file in files:
            with open(os.path.join(path, file), "r", encoding='utf-8') as f2:
                f.write(f2.read())
    print("完成合併, 檔案位於 %s " % target_file)


def texts2otr(path, target_file, audio_name, timeperiod):
    template = '''<p><span class="timestamp" data-timestamp="{}.000000">{}</span>{}</p><p><br/></p>
    '''
    files = os.listdir(path)
    content = ''
    files = [path+"/" + f for f in files if f.endswith(".txt")]
    with open(target_file, "w", encoding="utf-8") as f:

        for file in files:
            with open(file, "r", encoding="utf-8") as f2:
                txt = f2.read().split("\n")
                if len(txt) < 2:
                    continue
                times = int(txt[0].split("-")[1][:-5])*30
                secs, mins = times % 60, (times//60) % 60
                hours = (times//60)//60
                timeF = "{:02d}:{:02d}:{:02d}".format(hours, mins, secs)
                content += template.format(times, timeF, txt[1])

        output = {"text": content, "media": audio_name,
                  "media-time": timeperiod}
        f.write(json.dumps(output, ensure_ascii=False))



#@markdown 若 wav_path, txt_path 目錄存在是否移除重建
remove_dir='n' #@param ["y","n"]

files = os.listdir(wav_path)
VoiceToText(wav_path, files, txt_path)

target_txtfile = "{}{}.txt".format(path, FileName[:-4])
texts_to_one(txt_path, target_txtfile)
otr_file = "{}{}.otr".format(path, FileName[:-4])
with wave.open(FileName, "rb") as f:
    params = f.getparams()
texts2otr(txt_path, otr_file, FileName, params.nframes)
if remove_dir=='y':
  shutil.rmdir(wav_path)
  shutil.rmdir(txt_path)


210121_001-012.wav
这会是这一个你就是需求跟任务矩阵


KeyboardInterrupt: ignored

In [22]:
print("我是誰")

我是誰
